In [1]:
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_columns', None)

c:\Users\banfi\anaconda3\envs\pytorch-env\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Load Dataset

In [4]:
task = "task1"
SAVE_PATH = "data/"+task+"/"


In [5]:
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)


In [6]:
def load_data(task):
    df = pd.read_csv(task+"_data.txt")
    if task == "task2":
        train_labels = pd.read_csv(task+"_train_label.txt", header=None, sep="\t", names=["order","product", "label"])
        valid_labels = pd.read_csv(task+"_valid_label.txt", header=None, sep="\t", names=["order", "product","label"])
        test_query = pd.read_csv(task+"_test_query.txt",header=None,sep="\t", names=["order", "product"])
    else:
        train_labels = pd.read_csv(task+"_train_label.txt", header=None, sep="\t", names=["order", "label"])
        valid_labels = pd.read_csv(task+"_valid_label.txt", header=None, sep="\t", names=["order", "label"])
        test_query = pd.read_csv(task+"_test_query.txt",header=None,sep="\t", names=["order"])
    test_query["label"] = -1
    return df, train_labels, valid_labels, test_query

In [7]:
df = pd.read_csv(task+"_data.txt")

In [8]:

df, train_labels, valid_labels, test_query = load_data(task=task)
# labels = pd.concat([train_labels, valid_labels, test_query])


In [9]:
df

,order,product,customer,color,size,group
0,298957,54654,192219,169,10,11
1,298957,57127,192219,611,10,11
2,570617,23677,322757,43,24,19
3,570617,54993,322757,543,24,19
4,529565,10260,212452,92,17,19
...,...,...,...,...,...,...
2666257,117952,26468,81215,192,24,8
2666258,435829,21960,270329,31,10,11
2666259,435829,58150,270329,548,17,11
2666260,435829,20653,270329,437,17,8


In [10]:
def split_df(df, labels,train_labels, valid_labels, test_query):
    train_df = df.merge(train_labels)
    valid_df = df.merge(valid_labels)
    test_df = df.merge(test_query)
    df = df.merge(labels)

    return df,train_df, valid_df, test_df

In [11]:
def create_masks(labels,task):
    total_length = len(labels)
    train_masks = np.concatenate([np.ones(len(train_labels)), np.zeros(total_length - len(train_labels))])
    valid_masks = np.concatenate([np.zeros(len(train_labels)), np.ones(len(valid_labels)), np.zeros(len(test_query))])
    test_masks = np.concatenate([np.zeros(total_length - len(test_query)), np.ones(len(test_query))])

    if task == "task2":
        masks = pd.DataFrame({"order": labels["order"],"product": labels["product"],"train_masks": train_masks, "valid_masks": valid_masks, "test_masks": test_masks})
    else:
        masks = pd.DataFrame({"order": labels["order"],"train_masks": train_masks, "valid_masks": valid_masks, "test_masks": test_masks})
    return masks

In [12]:
labels = pd.concat([train_labels, valid_labels, test_query])
masks = create_masks(labels,task)
df,train_df, valid_df, test_df = split_df(df, labels,train_labels, valid_labels, test_query)


In [14]:
df

,order,product,customer,color,size,group,label
0,298957,54654,192219,169,10,11,0
1,298957,57127,192219,611,10,11,0
2,570617,23677,322757,43,24,19,1
3,570617,54993,322757,543,24,19,1
4,529565,10260,212452,92,17,19,0
...,...,...,...,...,...,...,...
2666257,117952,26468,81215,192,24,8,2
2666258,435829,21960,270329,31,10,11,0
2666259,435829,58150,270329,548,17,11,0
2666260,435829,20653,270329,437,17,8,0


# Create Customer Edgelist

In [17]:
def get_customer_edgelist(df: pd.DataFrame, masks, task) -> pd.DataFrame:
    """
    Returns the customer edgelist with the following edge attributes: customer, order, n_products, label
    
    Args:
        df (pandas.DataFrame): DataFrame containing customer order information.
    Returns:
        pandas.DataFrame: DataFrame containing customer edgelist.
    """
    # Group by customer
    if task == "task2":
        customer_edge_list = df.groupby(['order', 'customer']).size().reset_index().rename(columns={0: 'n_products'})
        customer_edge_list = customer_edge_list[['customer', 'order', 'n_products']].sort_values(by='customer')
    else:
        customer_edge_list = df.groupby(['order', 'customer', 'label']).size().reset_index().rename(columns={0: 'n_products'})
        customer_edge_list = customer_edge_list[['customer', 'order', 'n_products', 'label']].sort_values(by='customer')
        customer_edge_list = customer_edge_list.merge(masks,  on="order")

    return  customer_edge_list

def save_customer_edgelist(df: pd.DataFrame,masks,task, filename: str) -> None:
    """
    Saves the customer edgelist to a file.
    
    Args:
        df (pandas.DataFrame): DataFrame containing customer edgelist.
        filename (str): Filename to save the customer edgelist to.
    """
    output = get_customer_edgelist(df,masks, task)
    output.to_csv(filename, sep=',', index=False)
    return output

In [18]:
# customer_edge_list = save_customer_edgelist(df, masks, task,SAVE_PATH+'customer_edgelist.csv')
customer_edge_list = get_customer_edgelist(df, masks, task)

In [19]:
customer_edge_list

,customer,order,n_products,label,train_masks,valid_masks,test_masks
0,0,679928,1,-1,0.0,0.0,1.0
1,1,155095,4,1,1.0,0.0,0.0
2,1,390185,1,2,1.0,0.0,0.0
3,1,683973,2,-1,0.0,0.0,1.0
4,1,71090,5,2,1.0,0.0,0.0
...,...,...,...,...,...,...,...
849180,342035,770601,2,1,1.0,0.0,0.0
849181,342036,582026,8,1,0.0,1.0,0.0
849182,342037,793714,7,1,0.0,1.0,0.0
849183,342037,190206,3,-1,0.0,0.0,1.0


In [21]:
df[df["customer"] == 1]

,order,product,customer,color,size,group,label
326675,390185,31742,1,367,1,16,2
455241,155095,35837,1,573,1,11,1
455242,155095,42752,1,330,1,11,1
455243,155095,25483,1,202,1,11,1
455244,155095,21452,1,84,1,6,1
889282,71090,36584,1,348,1,11,2
889283,71090,57501,1,435,1,11,2
889284,71090,6115,1,348,1,19,2
889285,71090,12802,1,202,1,14,2
889286,71090,45776,1,136,1,14,2


# Create Order Edgelist

In [22]:
def get_order_edgelist(df: pd.DataFrame,task, masks) -> pd.DataFrame:
    """
    Returns the order edgelist with the following edge attributes: order, product, n_customers, label
    
    Args:
        df (pandas.DataFrame): DataFrame containing customer order information.
    Returns:
        pandas.DataFrame: DataFrame containing order edgelist.
    """
    # Group by order
    if task == "task2":
        order_groups = df.groupby(['order', 'product', 'label','color','size']).size().reset_index().rename(columns={0: 'n_customers'})
        order_groups = order_groups.merge(masks)
        return order_groups


        
    else:
        return order_groups[['order', 'product', 'label','color','size']].sort_values(by='order')

def save_order_edgelist(df: pd.DataFrame,task, masks, filename: str) -> None:
    """
    Saves the order edgelist to a file.
    
    Args:
        df (pandas.DataFrame): DataFrame containing order edgelist.
        filename (str): Filename to save the order edgelist to.
    """
    output = get_order_edgelist(df,task, masks)
    output.to_csv(filename, sep=',', index=False)

In [23]:
# save_order_edgelist(train_df, 'train_order_edgelist.csv')
# save_order_edgelist(df,task,masks, SAVE_PATH+'order_edgelist.csv')
xd = get_order_edgelist(df,task, masks)
xd

,order,product,label,color,size
0,0,18312,1,611,10
1,0,37434,1,611,17
2,0,40372,1,398,10
3,0,46793,1,485,17
4,1,11033,2,202,22
...,...,...,...,...,...
2666257,849182,56213,0,313,22
2666258,849183,44265,0,634,22
2666259,849183,53461,0,406,17
2666260,849184,20178,0,269,24


# Create Customer Node Attributes
Possibly find some way to find the average size of product ordered, but also linked to product type.

In [58]:
def get_customer_node_attributes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns the customer node attributes with the following attributes: customer, n_orders, n_products, label
    
    Args:
        df (pandas.DataFrame): DataFrame containing customer order information.
    Returns:
        pandas.DataFrame: DataFrame containing customer node attributes.
    """
    # Group by customer
    customer_groups = df.groupby(['customer', 'order', 'label']).size().reset_index().rename(columns={0: 'n_products'})
    customer_groups['n_not_returned'] = customer_groups['label'].apply(lambda x: 1 if x == 0 else 0)
    customer_groups['n_partial_returned'] = customer_groups['label'].apply(lambda x: 1 if x == 1 else 0)
    customer_groups['n_fully_returned'] = customer_groups['label'].apply(lambda x: 1 if x == 2 else 0)
    customer_groups = customer_groups.groupby('customer').agg({'order': 'count', 'n_not_returned': 'sum', 'n_partial_returned': 'sum', 'n_fully_returned': 'sum', 'n_products': 'sum'}).reset_index().rename(columns={'order': 'n_orders'})
    return customer_groups

def save_customer_node_attributes(df: pd.DataFrame, filename: str) -> None:
    """
    Saves the customer node attributes to a file.
    
    Args:
        df (pandas.DataFrame): DataFrame containing customer node attributes.
        filename (str): Filename to save the customer node attributes to.
    """
    output = get_customer_node_attributes(df)
    output.to_csv(filename, sep=',', index=False)

In [59]:
save_customer_node_attributes(df, SAVE_PATH+'customer_node_attributes.csv')

# Create Order Node Attributes

In [60]:
def get_order_node_attributes(df: pd.DataFrame, task) -> pd.DataFrame:
    """
    Returns the order node attributes with the following attributes: customer, n_orders, n_products, label
    
    Args:
        df (pandas.DataFrame): DataFrame containing customer order information.
    Returns:
        pandas.DataFrame: DataFrame containing customer node attributes.
    """
    if task == "task2":
        customer_groups = df.groupby(['order']).size().reset_index().rename(columns={0: 'n_products'})
    else:
        customer_groups = df.groupby(['order', 'label']).size().reset_index().rename(columns={0: 'n_products'})
    return customer_groups

def save_order_node_attributes(df: pd.DataFrame,task, filename: str) -> None:
    """
    Saves the order node attributes to a file.
    
    Args:
        df (pandas.DataFrame): DataFrame containing order node attributes.
        filename (str): Filename to save the order node attributes to.
    """
    output = get_order_node_attributes(df,task)
    output.to_csv(filename, sep=',', index=False)

In [61]:
# save_order_node_attributes(train_df, 'train_order_node_attributes.csv')
save_order_node_attributes(df,task, SAVE_PATH+'order_node_attributes.csv')

# Create Product Node Attributes

In [34]:
#todo: Finish this
def get_product_node_attributes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the node attributes for each product in the given DataFrame.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing the product data.

    Returns:
    pd.DataFrame: A DataFrame with the calculated node attributes for each product.
    """
    # Function implementation goes here

    # Group by customer
    product_stats = df.groupby('product').agg(
                                                total_count=('product', 'count'),
                                                return_1=('label', lambda x: (x==1).sum()),
                                                return_2=('label', lambda x: (x==2).sum())
                                                ).reset_index(drop=True)
    product_stats['return1_rate'] = product_stats['return_1'] / product_stats['total_count']
    product_stats['return2_rate'] = product_stats['return_2'] / product_stats['total_count']
    product_stats.rename_axis('product', inplace=True)
    return product_stats

def save_product_node_attributes(df: pd.DataFrame, filename: str) -> pd.DataFrame:
    """
    Save the product node attributes to a CSV file.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing the product node attributes.
    - filename (str): The name of the CSV file to save the attributes to.

    Returns:
    - pd.DataFrame: The DataFrame containing the product node attributes.

    """
    output = get_product_node_attributes(df)
    output.to_csv(filename, index=True)



In [35]:
get_product_node_attributes(df)

,total_count,return_1,return_2,return1_rate,return2_rate
product,,,,,
0,27,16,1,0.592593,0.037037
1,40,17,6,0.425000,0.150000
2,52,24,12,0.461538,0.230769
3,52,30,2,0.576923,0.038462
4,18,15,1,0.833333,0.055556
...,...,...,...,...,...
58410,11,7,4,0.636364,0.363636
58411,40,19,6,0.475000,0.150000
58412,39,22,2,0.564103,0.051282


In [63]:
# Returns calculated only on training data
save_product_node_attributes(train_df, SAVE_PATH+"product_node_attributes.csv")
# pa = get_product_node_attributes(train_df)